In [95]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="",
    api_key="",
)

/home/skeptrune/.local/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:115: UserWarning: Api key is used with unsecure connection.
  warnings.warn("Api key is used with unsecure connection.")


In [96]:
import clickhouse_connect

clickhouse_client = clickhouse_connect.get_client()
clickhouse_client.server_version

'22.10.7.13'

In [97]:
add_unique_constraint_query = """
DROP TABLE IF EXISTS qdrant_point_ids;
"""

clickhouse_client.query(add_unique_constraint_query)

create_table_query = """
CREATE TABLE IF NOT EXISTS qdrant_point_ids
(
    `point_id` String,
    `embedding_vector` Array(Float32)
)
ENGINE = MergeTree
ORDER BY point_id
"""

clickhouse_client.query(create_table_query)

In [98]:
offset = ""

while offset is not None:
    scroll_result = qdrant_client.scroll(
        collection_name="debate_cards",
        limit=100,
        with_vectors=True,
        offset=(offset if offset else None),
    )

    offset = scroll_result[1]

    insert_found_points_query = """
        INSERT INTO qdrant_point_ids
        VALUES
        """
    
    for point in scroll_result[0]:
        insert_found_points_query += f"('{point.id}', {point.vector}),"

    clickhouse_client.query(insert_found_points_query)

In [ ]:
get_qdrant_point_ids_count_query = """
SELECT *
FROM qdrant_point_ids
LIMIT 1
"""

select_result = clickhouse_client.query(get_qdrant_point_ids_count_query).result_rows

first_vector = select_result[0][1]

In [ ]:
cosine_similarity_query = f"""
SELECT
    point_id,
    cosineDistance(embedding_vector, {first_vector}) AS score
FROM qdrant_point_ids
ORDER BY score DESC
LIMIT 10
"""

select_result = clickhouse_client.query(cosine_similarity_query).result_rows

select_result

[('01de1f19-b8d3-4087-adc2-7b4179aac558', 0.32473153561611745),
 ('0091b309-5094-49de-a851-12987c568c84', 0.323417334853784),
 ('0091b309-5094-49de-a851-12987c568c84', 0.323417334853784),
 ('017bc8af-fe04-491e-b48a-dd1299eca7f5', 0.3233861418581041),
 ('0022ee7f-31ed-4619-bf70-80a05f37651a', 0.3232490557283646),
 ('0022ee7f-31ed-4619-bf70-80a05f37651a', 0.3232490557283646),
 ('02370150-9d76-499e-afc0-3b39c3e254ae', 0.322509401374503),
 ('003d04c8-a004-4817-a7cc-ddb5d5e3e0f2', 0.31961699619692663),
 ('003d04c8-a004-4817-a7cc-ddb5d5e3e0f2', 0.31961699619692663),
 ('017e5848-1039-44a1-9a3a-2c80fdac2aaa', 0.31956199618951153)]